#Asdex Upgrade Transfer Matrix

In [1]:
using VSTom

In [2]:
a = read_ncdf("data/full_synthetic_transfer_matrix.cdf")

transfer_matrix = a["transfer_matrix"]

energy = a["E"] #[keV]
ne_orig =length(energy)
we = energy .< 100.0
energy = energy[we]
dE = abs(energy[2] - energy[1])
nen = length(energy)

pitch = a["p"]
dP = abs(pitch[2] - pitch[1])
np = length(pitch);


nr,nc = size(transfer_matrix)
A = zeros(nr,np*nen)
for i=1:nr
    A[i,:] = reshape(reshape(transfer_matrix[i,:],np,ne_orig)[:,we],1,nen*np)
end

w1 = energy .< 10.0
inds=zeros(Bool,nr)
tol = 0.01 # 1%
for i=1:nr
    wf = reshape(A[i,:],np,nen)
    wf_total = sum(wf)
    if (sum(wf[:,w1])/wf_total < tol) 
        inds[i] = true 
    end
end
A = A[inds,:];

energy_red = linspace(20.0,maximum(energy),sum(energy .> 20.0))

A = resize_transfer_matrix(A,energy,pitch,energy_red,pitch)

1180x3000 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0        0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0        0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0        0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0        0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0        0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0        0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0        0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0        0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0        0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0        0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0        0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0        0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0        0.0  0.0  0.0  0.0
 ⋮        

In [3]:
length(energy_red)*length(pitch)

3000

In [6]:
using NetCDF

nrows, ncols = size(A)

# Write transfer matrix to file
fname = "data/reduced_synthetic_transfer_matrix.cdf"

isfile(fname) && rm(fname)

nrows_id = NcDim("nrows",nrows)
ncols_id = NcDim("ncols",ncols)
nen_id = NcDim("energy",length(energy_red))
np_id = NcDim("pitch",np)

t_varid = NcVar("transfer_matrix",[nrows_id,ncols_id])
e_varid = NcVar("energy",nen_id)
p_varid = NcVar("pitch",np_id)

ncid = NetCDF.create(fname,[t_varid,e_varid,p_varid],mode=NC_CLASSIC_MODEL)

NetCDF.putvar(ncid,"transfer_matrix",A)
NetCDF.putvar(ncid,"energy",energy_red)
NetCDF.putvar(ncid,"pitch",pitch)

NetCDF.close(ncid);